In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append("../")

import pandas as pd
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()

kv=ws.get_default_keyvault()


In [2]:
import os
dataset = Dataset.get_by_name(ws, name='test images')
mount_ctx = dataset.mount()
mount_ctx.start()
dataset_mount_folder = mount_ctx.mount_point
files = os.listdir(dataset_mount_folder)
files

UserErrorException: UserErrorException:
	Message: Mount is only supported on Unix or Unix-like operating systems with the native package libfuse installed. For more information, please refer to the remarks section of FileDataset.mount's documentation. Execution failedunexpectedly due to OSError
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Mount is only supported on Unix or Unix-like operating systems with the native package libfuse installed. For more information, please refer to the remarks section of FileDataset.mount's documentation. Execution failedunexpectedly due to OSError"
    }
}

#### Prerequisite
If you use the provision feature, then just need a workspace ws object
If you need to create the cluster manually
    1. Create a service principal and secret (SP)
    2. Provision an ADX cluster and create a DB
    3. Add the SP to be contributor of the cluster
    4. pip install following packages: pip install --upgrade azure-mgmt-kusto azure-kusto-ingest azure-kusto-data azure-identity 



#### Provisioning resource


The following code provision ADX cluster and register neccessary information at the workspace's keyvault

In [ ]:
#Option to set parameters to a custom ADX cluster
# from monitoring import KV_SP_ID, KV_SP_KEY, KV_ADX_DB, KV_ADX_URI, KV_TENANT_ID
# tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
# #Application ID
client_id = "af883abf-89dd-4889-bdb3-1ee84f68465e"
#Client Secret, set it at your WS' keyvault with key name same as your client_id
client_secret = kv.get_secret(client_id)

cluster_uri = "https://adx02.westus2.kusto.windows.net" #URL of the ADX Cluster
# kv.set_secret(KV_SP_ID,client_id)
# kv.set_secret(KV_SP_KEY,client_secret)
# kv.set_secret(KV_ADX_DB,"db01")
# kv.set_secret(KV_ADX_URI,cluster_uri)
# kv.set_secret(KV_TENANT_ID,tenant_id)
db_name = "db01"


In [ ]:
from obs.management import set_adx_to_workspace
set_adx_to_workspace(ws, cluster_uri,db_name, client_id, client_secret)


#### Test Ingestion


In [ ]:
from obs.collector import Online_Collector
table_name = "isd_weather_test4" #new dataset

sample_pd_data = pd.read_parquet("data/test_data.parquet").head(10)
sample_pd_data['timestamp'] = sample_pd_data['datetime']
sample_pd_data.drop(['datetime'], inplace=True, axis=1)

online_collector = Online_Collector(table_name,ws=ws)


In [ ]:
scaled_df = cat_output2.reset_index().rename({0:"scaled_value"}, axis=1)
scaled_df

In [ ]:
online_collector.stream_collect_df(sample_pd_data)

In [ ]:
online_collector.batch_collect(sample_pd_data)

### Streaming Ingestion and Real Time Visualization

In [ ]:

# Ingest streaming data  asynchronously with internal buffering mechanism to lower impact to main scoring thread
streaming_table_name="streaming_test"
streaming_collector = Online_Collector(streaming_table_name,ws=ws)

import random
streaming_collector.start_logging_daemon(buffer_time=2, batch_size=10)

for run_id in ["r000001", "r000002", "r000003", "r000004", "r000005"]:
    for i in range(1000):
        for lr in ["0.001", "0.002"]:
            df = pd.DataFrame({ "timestamp":pd.to_datetime('today'), "lr":[lr],"metric1":[random.uniform(3,50)] })
            streaming_collector.stream_collect_df_queue(df)
# streaming_collector.stop_logging()



In [ ]:
def some_test_function():
    import json
    print("test function")
    print("update that")


In [ ]:
def register_func(func_name):
    import inspect
    source = inspect.getsource(func_name)
    with open("source.py", "w") as fp:
        fp.write(source)
register_func(some_test_function)

In [ ]:
import source
content = inspect.getsource(source)
con